In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import cv2

class GuidedFilter:
    
    def __init__(self, I, radius=5, epsilon=0.4):

        self._radius = 2 * radius + 1
        self._epsilon = epsilon
        self._I = self._toFloatImg(I)
        
        self._initFilter()

    def _toFloatImg(self, img):
        if img.dtype == np.float32:
            return img
        return ( 1.0 / 255.0 ) * np.float32(img)

    def _initFilter(self):
        I = self._I
        r = self._radius
        eps = self._epsilon

        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]

        self._Ir_mean = cv2.blur(Ir, (r, r))
        self._Ig_mean = cv2.blur(Ig, (r, r))
        self._Ib_mean = cv2.blur(Ib, (r, r))

        Irr_var = cv2.blur(Ir ** 2, (r, r)) - self._Ir_mean ** 2 + eps                                       
        Irg_var = cv2.blur(Ir * Ig, (r, r)) - self._Ir_mean * self._Ig_mean                                  
        Irb_var = cv2.blur(Ir * Ib, (r, r)) - self._Ir_mean * self._Ib_mean                                  
        Igg_var = cv2.blur(Ig * Ig, (r, r)) - self._Ig_mean * self._Ig_mean + eps                            
        Igb_var = cv2.blur(Ig * Ib, (r, r)) - self._Ig_mean * self._Ib_mean                                  
        Ibb_var = cv2.blur(Ib * Ib, (r, r)) - self._Ib_mean * self._Ib_mean + eps                                                                                     


        self._Ir_mean = cv2.blur(Ir, (r, r))
        self._Ig_mean = cv2.blur(Ig, (r, r))
        self._Ib_mean = cv2.blur(Ib, (r, r))

        Irr_var = cv2.blur(Ir ** 2, (r, r)) - self._Ir_mean ** 2 + eps                                       
        Irg_var = cv2.blur(Ir * Ig, (r, r)) - self._Ir_mean * self._Ig_mean                                  
        Irb_var = cv2.blur(Ir * Ib, (r, r)) - self._Ir_mean * self._Ib_mean                                  
        Igg_var = cv2.blur(Ig * Ig, (r, r)) - self._Ig_mean * self._Ig_mean + eps                            
        Igb_var = cv2.blur(Ig * Ib, (r, r)) - self._Ig_mean * self._Ib_mean                                  
        Ibb_var = cv2.blur(Ib * Ib, (r, r)) - self._Ib_mean * self._Ib_mean + eps                                                       


        Irr_inv = Igg_var * Ibb_var - Igb_var * Igb_var                                                      
        Irg_inv = Igb_var * Irb_var - Irg_var * Ibb_var                                                      
        Irb_inv = Irg_var * Igb_var - Igg_var * Irb_var                                                      
        Igg_inv = Irr_var * Ibb_var - Irb_var * Irb_var                                                      
        Igb_inv = Irb_var * Irg_var - Irr_var * Igb_var                                                      
        Ibb_inv = Irr_var * Igg_var - Irg_var * Irg_var                                                      
        
        I_cov = Irr_inv * Irr_var + Irg_inv * Irg_var + Irb_inv * Irb_var                                    
        Irr_inv /= I_cov                                                                                     
        Irg_inv /= I_cov                                                                                     
        Irb_inv /= I_cov                                                                                     
        Igg_inv /= I_cov                                                                                     
        Igb_inv /= I_cov                                                                                     
        Ibb_inv /= I_cov                                                                                     
        
        self._Irr_inv = Irr_inv                                                                              
        self._Irg_inv = Irg_inv                                                                              
        self._Irb_inv = Irb_inv                                                                              
        self._Igg_inv = Igg_inv                                                                              
        self._Igb_inv = Igb_inv                                                                              
        self._Ibb_inv = Ibb_inv                  

    def _computeCoefficients(self, p):
        r = self._radius                                                             
        I = self._I                                                                 
        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]                                                          
        

        p_mean = cv2.blur(p, (r, r))                             
        Ipr_mean = cv2.blur(Ir * p, (r, r))                                                         
        Ipg_mean = cv2.blur(Ig * p, (r, r))                                                    
        Ipb_mean = cv2.blur(Ib * p, (r, r))             



        Ipr_cov = Ipr_mean - self._Ir_mean * p_mean                                                 
        Ipg_cov = Ipg_mean - self._Ig_mean * p_mean                                                     
        Ipb_cov = Ipb_mean - self._Ib_mean * p_mean                                                       
                                                                                                                 
        ar = self._Irr_inv * Ipr_cov + self._Irg_inv * Ipg_cov + self._Irb_inv * Ipb_cov                 
        ag = self._Irg_inv * Ipr_cov + self._Igg_inv * Ipg_cov + self._Igb_inv * Ipb_cov                
        ab = self._Irb_inv * Ipr_cov + self._Igb_inv * Ipg_cov + self._Ibb_inv * Ipb_cov    

        b = p_mean - ar * self._Ir_mean - ag * self._Ig_mean - ab * self._Ib_mean                                                                                                                                         

        ar_mean = cv2.blur(ar, (r, r))          
        ag_mean = cv2.blur(ag, (r, r))                                                                   
        ab_mean = cv2.blur(ab, (r, r))                                                                      
        b_mean = cv2.blur(b, (r, r))                                                                                                                                              

        return ar_mean, ag_mean, ab_mean, b_mean            

    def _computeOutput(self, ab, I):
    
        ar_mean, ag_mean, ab_mean, b_mean = ab
        Ir, Ig, Ib = I[:, :, 0], I[:, :, 1], I[:, :, 2]
        q = ar_mean * Ir + ag_mean * Ig + ab_mean * Ib + b_mean
        return q

    def filter(self, p):
        
        p_32F = self._toFloatImg(p)
        ab = self._computeCoefficients(p)
        return self._computeOutput(ab, self._I)

In [ ]:
# Importing Libraries
import sys,os
import cv2
import numpy as np
import scipy
import scipy.ndimage
import matplotlib.pyplot as plt

import time
# !pip install imutils
import imutils
from imutils.video import VideoStream
# from google.colab.patches import cv2_imshow
# !mkdir output

# Defining important functions:

#from Image_Processing_Project import GuidedFilter

# Calculated Depth Map:
def calDepthMap(I, r):
    #Calculate the depth map

    hsvI = cv2.cvtColor(I, cv2.COLOR_BGR2HSV)
    s = hsvI[:,:,1] / 255
    v = hsvI[:,:,2] / 255
    #cv2.imshow("hsvI",hsvI)
    #cv2.waitKey()

    sigma = 0.04       # Smaller value of sigma gives more clearer image (tried 0.02) (0.04 Good)
    sigmaMat = np.random.normal(0, sigma, (I.shape[0], I.shape[1]))

    output1 =  0.12 + 0.959710 * v - 0.780245 * s + sigmaMat  # Increasign the value of 1st variable (theta_0) increases image quality (tried 0.2) (0.12 Good)
    outputPixel = output1
    output2 = scipy.ndimage.filters.minimum_filter(output1,(r,r))
    outputRegion = output2
    # cv2.imwrite("/content/output/vsFeature.jpg", outputRegion*255 )
    #cv2.imshow("outputRegion",outputRegion)
    #cv2.waitKey()
    return outputRegion, outputPixel


# Estimated Atmospheric Light:
def estA(img, Jdark):
    #Estimate the value of atmospheric ambient light A

    h,w,c = img.shape
    if img.dtype == np.uint8:
        img = np.float32(img) / 255
    
    # Compute number for 0.1% brightest pixels
    n_bright = int(np.ceil(0.001*h*w))
    #  Loc contains the location of the sorted pixels
    reshaped_Jdark = Jdark.reshape(1,-1)
    Y = np.sort(reshaped_Jdark) 
    Loc = np.argsort(reshaped_Jdark)
    
    # column-stacked version of I
    Ics = img.reshape(1, h*w, 3)
    ix = img.copy()
    dx = Jdark.reshape(1,-1)
    
    # init a matrix to store candidate airlight pixels
    Acand = np.zeros((1, n_bright, 3), dtype=np.float32)
    # init matrix to store largest norm arilight
    Amag = np.zeros((1, n_bright, 1), dtype=np.float32)
    
    # Compute magnitudes of RGB vectors of A
    for i in range(n_bright):
        x = Loc[0,h*w-1-i]
        # print(x,x/w,x%w)
        # ix[int(x/w), x%w, 0] = 0
        # ix[int(x/w), x%w, 1] = 0
        # ix[int(x/w), x%w, 2] = 1
        
        Acand[0, i, :] = Ics[0, Loc[0, h*w-1-i], :]
        Amag[0, i] = np.linalg.norm(Acand[0,i,:])
    
    # Sort A magnitudes
    reshaped_Amag = Amag.reshape(1,-1)
    Y2 = np.sort(reshaped_Amag) 
    Loc2 = np.argsort(reshaped_Amag)
    # A now stores the best estimate of the airlight
    if len(Y2) > 20:
        A = Acand[0, Loc2[0, n_bright-19:n_bright],:]
    else:
        A = Acand[0, Loc2[0,n_bright-len(Y2):n_bright],:]
    
    # finds the max of the 20 brightest pixels in original image
    #print(A)

    #cv2.imshow("brightest",ix)
    #cv2.waitKey()
    # cv2.imwrite("/content/output/position_of_the_atmospheric_light.png", ix*255)
    
    return A
def notFinal(I):
   save.write(np.uint8(J*255))

def final(I):
    # Other Variables for Image Processing:
    r = 10 #The size of the box when the minimum value is filtered
    beta = 1.0 #scattering coefficient
    gimfiltR = 40 #The size of the radius when guiding the filter
    eps = 10**-2 #The value of epsilon when guiding the filter (Increasing the value highlights more shadows in the image)
    
    dR,dP = calDepthMap(I, r)
    guided_filter = GuidedFilter(I, gimfiltR, eps)
    refineDR = guided_filter.filter(dR)
    tR = np.exp(-beta * refineDR)

    a = estA(I, dR)

    if I.dtype == np.uint8:
        I = np.float32(I) / 255

    h,w,c = I.shape
    J = np.zeros((h, w, c), dtype=np.float32)

    J[:,:,0] = I[:,:,0] - a[0,0]
    J[:,:,1] = I[:,:,1] - a[0,1]
    J[:,:,2] = I[:,:,2] - a[0,2]

    t = tR
    t0, t1 = 0.05, 1
    t = t.clip(t0, t1)

    J[:, :, 0] = (J[:, :, 0]  / t) + a[0, 0]
    J[:, :, 1] = (J[:, :, 1]  / t) + a[0, 1]
    J[:, :, 2] = (J[:, :, 2]  / t) + a[0, 2]
    # print(J.shape)
    frameR = np.uint8(J*255)
#     save.write(frameR)
    return frameR


if __name__ == "__main__":
    #parameter settings
    # inputImagePath = "/content/drive/Shareddrives/image processing/Online_Image/foggy_bench.jpg" #Enter image path
    
    # Video Input:
    # inputVideoPath = "/content/drive/Shareddrives/image processing/Youtube_Videos/Input Videos/Dubai skyscrapers view in winter - Foggy morning.mp4"
    # vidObj = cv2.VideoCapture(inputVideoPath)

#     vidObj_cv = cv2.VideoCapture(0)
    vidObj_vs = VideoStream(0).start()

    # Loop for frames:
    # fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    # frame_width = vidObj.get(cv2.CAP_PROP_FRAME_WIDTH)
    # frame_height = vidObj.get(cv2.CAP_PROP_FRAME_HEIGHT)
    # fps = vidObj.get(cv2.CAP_PROP_FPS)
#     frame_nos = vidObj.get(cv2.CAP_PROP_FRAME_COUNT)
    # save = cv2.VideoWriter('/content/output.mp4',fourcc, fps, (int(frame_width), int(frame_height)))

    # Just some extra details:
    # print('Frame Width:',frame_width)
    # print('Frame Height:',frame_height)
    # print('Number of Frames:',frame_nos)
    # print('FPS:',fps)
    ###
    # FI = None

    time.sleep(1.0)
    start_time = time.time()
    
    while True:
#         ret, I_cv = vidObj_cv.read()
        I_vs = vidObj_vs.read()
        
#         cv2.imshow('Input Sequence',I_cv)
        cv2.imshow('Input Sequence',I_vs)

#         if (frame_idx % 10 == 0) and ret:
#             FI=final(I)
#         else:
#             save.write(FI)

#         FI_cv = final(I_cv)
        FI_vs = final(I_vs)
        
#         cv2.imshow('Output Sequence',FI_cv)
        cv2.imshow('Output Sequence',FI_vs)

        # Breaking Loop:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

      # frame_idx+=10
    
#     vidObj_cv.release()
    vidObj_vs.stream.release()
    # save.release()
    cv2.destroyAllWindows()
    
# print('Execution Completed')

print("[INFO] Elasped Time: {:.2f} sec".format(time.time() - start_time))